In [1]:
import trimesh
from plyfile import PlyData, PlyElement, PlyProperty
import numpy as np
import laspy as lp

In [32]:
import numpy as np
import pandas as pd

In [3]:
# point_cloud=lp.read("davis.las")
# original_class = np.array(point_cloud.classification)

In [4]:
plydata = PlyData.read('predictions/bin_davis_008.ply')

In [5]:
plydata.elements

(PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('preds', 'int'), PlyProperty('pots', 'double')), count=2595810, comments=[]),)

In [6]:
plydata.elements[0].properties

(PlyProperty('x', 'float'),
 PlyProperty('y', 'float'),
 PlyProperty('z', 'float'),
 PlyProperty('preds', 'int'),
 PlyProperty('pots', 'double'))

In [7]:
plydata.elements[0].count

2595810

In [8]:
plydata['vertex']

PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('preds', 'int'), PlyProperty('pots', 'double')), count=2595810, comments=[])

In [9]:
point_data = np.stack([plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z'], plydata['vertex']['preds']], axis=0).transpose((1, 0))


In [10]:
point_data

array([[7.44434998e+02, 1.09817004e+03, 1.95235001e+02, 1.00000000e+00],
       [7.46505005e+02, 1.09818005e+03, 1.95215012e+02, 1.00000000e+00],
       [7.53794983e+02, 1.09825000e+03, 1.95085007e+02, 1.00000000e+00],
       ...,
       [4.88005005e+02, 1.07270996e+03, 2.07865005e+02, 8.00000000e+00],
       [4.90855011e+02, 1.07515002e+03, 2.06855011e+02, 8.00000000e+00],
       [4.90484985e+02, 1.07566003e+03, 2.06835007e+02, 8.00000000e+00]])

In [12]:
df = pd.DataFrame(data=point_data, columns=['X','Y','Z','Class'])
df

,X,Y,Z,Class
0,744.434998,1098.170044,195.235001,1.0
1,746.505005,1098.180054,195.215012,1.0
2,753.794983,1098.250000,195.085007,1.0
3,752.934998,1098.250000,198.425003,2.0
4,752.424988,1096.829956,200.645004,2.0
...,...,...,...,...
2595805,489.815002,1070.910034,204.195007,1.0
2595806,488.044983,1071.250000,205.285004,8.0
2595807,488.005005,1072.709961,207.865005,8.0
2595808,490.855011,1075.150024,206.855011,8.0


In [13]:
df.Class.value_counts()

1.0    1406072
2.0     736200
8.0     447463
4.0       5236
5.0        473
3.0        276
7.0         90
Name: Class, dtype: int64

In [14]:
threshold = df.Z.where(df.Class == 8)
threshold.quantile(0.35)

201.4250030517578

In [15]:
df['Class'] = np.where((df['Class'] ==1) & (df['Z'] > threshold.quantile(0.4)), 8, df['Class'])
df.Class.value_counts()

1.0    1333171
2.0     736200
8.0     520364
4.0       5236
5.0        473
3.0        276
7.0         90
Name: Class, dtype: int64

In [16]:
# df['o_class'] = original_class
# df

In [17]:
# df['o_class'].value_counts()

In [18]:
# df['Class'] = np.where(df['o_class']==1, 1, df['Class'])
# df['Class'] = np.where((df['Class'] ==1) & (df['o_class']!=1), 8, df['Class'])


In [19]:
# df.Class.value_counts()

In [20]:
# df = df.drop('o_class', axis=1)

In [21]:
df['r']= 0.0
df['g']= 0.0
df['b']= 0.0

In [22]:
df

,X,Y,Z,Class,r,g,b
0,744.434998,1098.170044,195.235001,1.0,0.0,0.0,0.0
1,746.505005,1098.180054,195.215012,1.0,0.0,0.0,0.0
2,753.794983,1098.250000,195.085007,1.0,0.0,0.0,0.0
3,752.934998,1098.250000,198.425003,2.0,0.0,0.0,0.0
4,752.424988,1096.829956,200.645004,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2595805,489.815002,1070.910034,204.195007,8.0,0.0,0.0,0.0
2595806,488.044983,1071.250000,205.285004,8.0,0.0,0.0,0.0
2595807,488.005005,1072.709961,207.865005,8.0,0.0,0.0,0.0
2595808,490.855011,1075.150024,206.855011,8.0,0.0,0.0,0.0


In [23]:
df['Class'].value_counts()

1.0    1333171
2.0     736200
8.0     520364
4.0       5236
5.0        473
3.0        276
7.0         90
Name: Class, dtype: int64

In [24]:
%%time
import warnings
warnings.filterwarnings('ignore')
for idx, x in enumerate(df['Class']):
    if x == 1.0: ## groud Blue
        df['b'][idx] = 95
        # df['r'][idx] = 173
        # df['g'][idx] = 216
        # df['b'][idx] = 230
    elif x == 2.0: ## veg Dark Green
        df['r'][idx] = 2
        df['g'][idx] = 50
        df['b'][idx] = 35
    elif x == 3.0: ## cars Pink
        df['r'][idx] = 255
        df['g'][idx] = 105
        df['b'][idx] = 180
    elif x == 4.0: ## truck Yellow
        df['r'][idx] = 255
        df['g'][idx] = 255
    elif x == 5.0: ## pole line Green
        df['g'][idx] = 255
    elif x == 6.0: ## fences Light Blue
        df['r'][idx] = 164
        df['g'][idx] = 219
        df['b'][idx] = 232
    elif x == 7.0: ## pole Orange
        df['r'][idx] = 255
        df['g'][idx] = 172
    elif x == 8.0: ## house Red
        df['r'][idx] = 200
    # elif x == 9.0: ## undefined Black
        # df['r'][idx] = 0
        # df['g'][idx] = 0
        # df['b'][idx] = 0

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


CPU times: user 3min 28s, sys: 10.8 s, total: 3min 39s
Wall time: 3min 30s


In [25]:
arr = df.to_numpy()

In [26]:
some_arr = np.array([tuple(i) for i in arr], dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'),('class', 'f4'),('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

In [27]:
some_arr

array([(744.435, 1098.17, 195.235  , 1.,   0, 0, 95),
       (746.505, 1098.18, 195.21501, 1.,   0, 0, 95),
       (753.795, 1098.25, 195.085  , 1.,   0, 0, 95), ...,
       (488.005, 1072.71, 207.865  , 8., 200, 0,  0),
       (490.855, 1075.15, 206.85501, 8., 200, 0,  0),
       (490.485, 1075.66, 206.835  , 8., 200, 0,  0)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('class', '<f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

In [28]:
el = PlyElement.describe(some_arr, 'vertex')

In [29]:
PlyData([el], text=True).write('davis_008_visual.ply')

In [30]:
100 % 1

0

In [31]:
len(arr)

2595810